In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, LSTM, Subtract
from tensorflow.keras.models import Model
from Environment import *
from tqdm import tqdm
import matplotlib.pyplot as plt
from copy import deepcopy
import os 
import numpy as np

In [2]:
dense_1_user =  Dense(32, activation = 'relu')
dense_2_user =  Dense(32, activation = 'relu')
# dense_3_user =  Dense(32, activation = 'relu')

dense_1_assist =  Dense(32, activation = 'relu')
lstm_1_assist = LSTM(32, activation = 'tanh')
dense_2_assist = Dense(32, activation = 'relu')

advantage_layer_user = Dense(4)
value_layer_user = Dense(1)

advantage_layer_assist = Dense(4)
value_layer_assist = Dense(1)

# advantage_layer = Dense(4)
# value_layer = Dense(1)

# advantage_layer_user = advantage_layer
# advantage_layer_assist = advantage_layer

# value_layer_user = value_layer
# value_layer_assist = value_layer

In [3]:
class AI_Design:
    def __init__(self, steps = 4):        
        self.loss_fn = tf.keras.losses.mean_squared_error
        self.optimizer = tf.keras.optimizers.Adam(lr = 0.0001)
        self.batch_size = 128
        self.replay_buffer_size = 1024
        self.replay_buffer = Replay_Buffer(self.replay_buffer_size)
        self.epsilon = 1
        self.gamma = 0.9
        self.env = Environment()
        self.env.cells = np.array([[0.7, 0.1], [0.1, 0.1], [0.5, 0.7], [0.6, 0.2], [0.7, 0.4], [0.2, 0.9]])
        self.env_cell_mapping = give_mapping(self.env.cells)
        self.env_cell_mapping = self.env_cell_mapping[np.newaxis, :, :, np.newaxis]
        #-------------------------------------------------------------------------------------------------
        input_A = Input(shape = (4,))
        input_B = Input(shape = (steps,6))
        input_C = Input(shape = (11, 11, 1)) #Location of every icon
                 
        action_user = Input(shape = 1, dtype = tf.int32)
        action_assist = Input(shape = 1, dtype = tf.int32)
        
        
        
        #User Network 
        x = Subtract()([input_A[:, 2:], input_A[:, :2]])
        x = dense_1_user(x)
        x = dense_2_user(x)
#         x = dense_3_user(x)
        adv_user = advantage_layer_user(x)
        val_user = value_layer_user(x)
        output_user = adv_user - tf.reduce_mean(adv_user, axis = 1, keepdims = True) + val_user
        
        self.user_model = Model(inputs = input_A, outputs = output_user)
        self.user_model.summary()
        
        self.target_user_model = tf.keras.models.clone_model(self.user_model)
        self.target_user_model.set_weights(self.user_model.get_weights())
        

        
        #Assistant Network
        z = tf.keras.layers.Conv2D(filters = 2, kernel_size = 3, activation = 'relu')(input_C)
        z = tf.keras.layers.MaxPooling2D()(z)
        z = tf.keras.layers.Flatten()(z)
        z = tf.keras.layers.Dense(32, activation = 'relu')(z)
        
        y = dense_1_assist(input_B)
        y = lstm_1_assist(y)
        y = tf.keras.layers.Concatenate()([y,z])
        y = dense_2_assist(y)
        adv_assist = advantage_layer_assist(y)
        val_assist = value_layer_assist(y)
        output_assist = adv_assist - tf.reduce_mean(adv_assist, axis = 1, keepdims = True) + val_assist
        
        self.assist_model = Model(inputs = [input_B, input_C], outputs = output_assist)
        self.assist_model.summary()
        
        self.target_assist_model = tf.keras.models.clone_model(self.assist_model)
        self.target_assist_model.set_weights(self.assist_model.get_weights())
        
        
        #Complete Network
        mask_user = tf.reduce_sum(tf.one_hot(action_user, 4), axis = 1)
        mask_assist = tf.reduce_sum(tf.one_hot(action_assist, 4), axis = 1)
        output_user = output_user*mask_user
        output_assist = output_assist*mask_assist
        
        out = tf.reduce_sum(output_user + output_assist, axis = 1, keepdims = True)
        
        self.model = Model(inputs = [input_A, input_B, input_C, action_user, action_assist], outputs = out)  
        self.model.summary() 
        #-------------------------------------------------------------------------------------------------
    
    def infer(self):
        ob_user, action_user, reward_user, next_ob_user, ob_assist, action_assist,\
        reward_assist, next_ob_assist, done, importance, indices = self.sample_exp()
        
        ob_user = ob_user[:4]
        action_user = action_user[:4]
        reward_user = reward_user[:4]
        
        ob_assist = ob_assist[:4]
        action_assist = action_assist[:4]
        reward_assist = reward_assist[:4]
        
        print(action_user, action_assist)
        
        print(self.user_model(ob_user))
        print(self.assist_model([ob_assist, np.vstack(tuple([self.env_cell_mapping]*4))]))
        
        print(self.model([ob_user, ob_assist, np.vstack(tuple([self.env_cell_mapping]*4)), action_user, action_assist]))
    
    def exp_policy_user(self, state, next_action = False):
        if np.random.rand()<self.epsilon:
            return np.random.randint(4)
        else:
            state = np.array(state)[np.newaxis]
            if next_action == False:
                Q_values = self.user_model(state)
            else:
                Q_values = self.target_user_model(state)
            return np.argmax(Q_values[0])
    
    def exp_policy_assist(self, state):
        if np.random.rand()<self.epsilon:
            return np.random.randint(1,5)
        else:
            state = np.array(state)[np.newaxis]
            Q_values = self.assist_model([state, self.env_cell_mapping])
            return np.argmax(Q_values[0])+1
    
    def step(self, ob_user, prev_steps_assist):
        curr_loc = ob_user[:2]
        target_loc = ob_user[2:4]
        
        action_user = self.exp_policy_user(ob_user)
        action_user_one_hot = make_one_hot(action_user, 4)
        
        ob_assist = [action_user_one_hot + ob_user[:2]]
        ob_assist = prev_steps_assist + ob_assist 
        action_assist = self.exp_policy_assist(ob_assist)
        
        new_loc, reward_user, reward_assist, done = self.env.step(action_user, action_assist, target_loc, curr_loc)
        
        next_ob_user = new_loc[:]
        next_ob_user = next_ob_user + target_loc
        
        next_action_user = self.exp_policy_user(next_ob_user, next_action = True)
        next_action_user_one_hot = make_one_hot(next_action_user, 4)
        next_ob_assist = [next_action_user_one_hot + next_ob_user[:2]]
        next_ob_assist = ob_assist[1:] + next_ob_assist
        
        self.add_replay_buffer(ob_user, action_user, reward_user, next_ob_user, ob_assist,\
                          action_assist-1, reward_assist, next_ob_assist, done)
        
        return next_ob_user, ob_assist[1:], reward_user, reward_assist, done 
        
        
    
    def add_replay_buffer(self, ob_user, action_user, reward_user, next_ob_user, ob_assist,\
                         action_assist, reward_assist, next_ob_assist, done):
        
        self.replay_buffer.ob_user_history.append(ob_user)
        self.replay_buffer.action_user_history.append(action_user)
        self.replay_buffer.reward_user_history.append(reward_user)
        self.replay_buffer.next_ob_user_history.append(next_ob_user)
        self.replay_buffer.ob_assist_history.append(ob_assist)
        self.replay_buffer.action_assist_history.append(action_assist)
        self.replay_buffer.reward_assist_history.append(reward_assist)
        self.replay_buffer.next_ob_assist_history.append(next_ob_assist)
        self.replay_buffer.done_history.append(done)
        self.replay_buffer.priorities.append(self.replay_buffer.max_val)
    
    def sample_exp(self):
        sample_probs = self.replay_buffer.get_probabilities(priority_scale = 0.7)
        indices = np.random.choice(len(self.replay_buffer.done_history), size = self.batch_size, p = sample_probs)
        importance = self.replay_buffer.get_importance(sample_probs[indices])
        
        ob_user = np.array([self.replay_buffer.ob_user_history[i] for i in indices])
        action_user = np.array([self.replay_buffer.action_user_history[i] for i in indices])
        reward_user = np.array([self.replay_buffer.reward_user_history[i] for i in indices])
        next_ob_user = np.array([self.replay_buffer.next_ob_user_history[i] for i in indices])
        ob_assist = np.array([self.replay_buffer.ob_assist_history[i] for i in indices])
        action_assist = np.array([self.replay_buffer.action_assist_history[i] for i in indices])
        reward_assist = np.array([self.replay_buffer.reward_assist_history[i] for i in indices])
        next_ob_assist = np.array([self.replay_buffer.next_ob_assist_history[i] for i in indices])
        done = np.array([self.replay_buffer.done_history[i] for i in indices])
        
        return ob_user, action_user, reward_user, next_ob_user, ob_assist, action_assist, reward_assist, next_ob_assist, done,\
    importance, indices 
    
    def train(self):
        ob_user, action_user, reward_user, next_ob_user, ob_assist, action_assist,\
        reward_assist, next_ob_assist, done, importance, indices = self.sample_exp()
        
        input_A = ob_user
        input_B = ob_assist
        input_C = np.vstack(tuple([self.env_cell_mapping]*128))
        
        rewards = reward_user + reward_assist
        
        next_Q_values_user, next_Q_values_assist = self.user_model(next_ob_user), self.assist_model([next_ob_assist, input_C])
        best_next_actions_user, best_next_actions_assist = tf.math.argmax(next_Q_values_user, axis = 1), tf.math.argmax(next_Q_values_assist, axis = 1)
        next_Q_values_user, next_Q_values_assist = self.target_user_model(next_ob_user), self.target_assist_model([next_ob_assist, input_C])
        
        best_next_Q_values_user = tf.reduce_sum(next_Q_values_user*tf.one_hot(best_next_actions_user, 4), axis = 1)
        best_next_Q_values_assist = tf.reduce_sum(next_Q_values_user*tf.one_hot(best_next_actions_assist, 4), axis = 1)
        best_next_Q_values = best_next_Q_values_user + best_next_Q_values_assist
        
        target_Q_values = rewards + (1-done)*self.gamma*best_next_Q_values
        
        with tf.GradientTape() as tape:
            Q_values = self.model([input_A, input_B, input_C, action_user, action_assist])
            error = tf.multiply(self.loss_fn(target_Q_values, Q_values), importance**(1-self.epsilon))
            loss = tf.reduce_mean(error)
        
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
        self.replay_buffer.set_priorities(indices, error)
        
        
#         Q_values_assist = self.assist_model([input_B, input_C])
#         with tf.GradientTape() as tape:
#             Q_values_user = self.user_model(input_A)
#             error = tf.multiply(self.loss_fn(Q_values_assist, Q_values_user), importance**(1-self.epsilon))
#             loss = tf.reduce_mean(error)
            
#         grads = tape.gradient(loss, self.user_model.trainable_variables)
#         self.optimizer.apply_gradients(zip(grads, self.user_model.trainable_variables))
            
#         with tf.GradientTape() as tape:
#             Q_values_assist = self.assist_model([input_B, input_C])
#             error = tf.multiply(self.loss_fn(Q_values_assist, Q_values_user), importance**(1-self.epsilon))
#             loss = tf.reduce_mean(error)
            
#         grads = tape.gradient(loss, self.assist_model.trainable_variables)
#         self.optimizer.apply_gradients(zip(grads, self.assist_model.trainable_variables))                          

        

In [4]:
steps = 4
model = AI_Design(steps)
env = model.env

# if os.path.exists('user_model.h5'):
#     model.user_model = tf.keras.models.load_model('user_model.h5')
#     model.assist_model = tf.keras.models.load_model('assist_model.h5')

Icon Locations:
[[0.8 0.2]
 [0.6 0. ]
 [0.  0.9]
 [0.4 0.9]
 [0.1 0.9]
 [0.1 0.3]]
Icon usage Probabilities
[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 2)]          0           input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (Te [(None, 2)]          0           input_1[0][0]                    
__________________________________________________________________________________________________
subtract (

In [5]:
def give_prev_steps(prev_steps_assist, steps):
    prev_steps_assist = [[0,0,0,0,-1,-1] for i in range(steps-1)]
    return prev_steps_assist

In [6]:
max_steps = 40
reached = 0
reached_history = []
max_reached = 0

running_reward = 0

for epoch in tqdm(range(100000)):
    done = False
    episode_reward = 0
    start, dest = env.give_start_dest()
    ob_user = [start[0], start[1], dest[0], dest[1]]
    prev_steps_assist = []
    prev_steps_assist = give_prev_steps(prev_steps_assist, steps)
    step = 0
    
    while not done and step<max_steps:
        ob_user, prev_steps_assist, reward_user, reward_assist, done = model.step(ob_user, prev_steps_assist)
        episode_reward+=reward_user
        step+=1
        if done:
            reached+=1
    
    if epoch:
        running_reward = 0.01 * episode_reward + (1 - 0.01) * running_reward
    else:
        running_reward = episode_reward
        
    if epoch>50:
        model.train()
        
        if epoch%100==0:
            model.target_user_model.set_weights(model.user_model.get_weights())
            model.target_assist_model.set_weights(model.assist_model.get_weights())
            reached_history.append(reached)
            rewards = []
            
            if reached>max_reached:
                print(reached)
                print('Saved Weights')
                max_reached = reached
                model.user_model.save('user_model.h5')
                model.assist_model.save('assist_model.h5')
                
#                 if reached == 100:
#                     print('Environment solved..........\nStopping training...........')
#                     break
                
            reached = 0
            
            if epoch%500==0:
                print(f'Running reward = {running_reward}')
                print(f'Successful runs = {np.mean(reached_history)}')
                reached_history = []
                
                if epoch%1000==0:
                    model.epsilon-=0.01
                    model.epsilon= max(model.epsilon, 0.1)
                    
                    if epoch%20000==0:
                        model.infer()

  0%|                                                                           | 101/100000 [00:13<5:03:04,  5.49it/s]

15
Saved Weights


  0%|▏                                                                          | 301/100000 [00:46<5:31:52,  5.01it/s]

23
Saved Weights


  1%|▍                                                                          | 502/100000 [01:23<4:48:14,  5.75it/s]

...

 99%|███████████████████████████████████████████████████████████████████████▎| 99001/100000 [10:45:31<12:14,  1.36it/s]

Running reward = -27.760859806918894
Successful runs = 25.0


100%|███████████████████████████████████████████████████████████████████████▋| 99501/100000 [10:51:19<06:34,  1.26it/s]

Running reward = -32.23396163993609
Successful runs = 19.8


100%|███████████████████████████████████████████████████████████████████████| 100000/100000 [10:57:23<00:00,  2.54it/s]


In [7]:
model.user_model = tf.keras.models.load_model('user_model.h5')
model.assist_model = tf.keras.models.load_model('assist_model.h5')

In [8]:
model.infer()

[3 1 1 1] [1 3 3 1]
tf.Tensor(
[[0.18308994 0.17637497 0.03688546 0.0847222 ]
 [0.10390993 0.06150039 0.00904826 0.04528475]
 [0.13619444 0.10421284 0.0895853  0.06934924]
 [0.18308994 0.17637497 0.03688546 0.0847222 ]], shape=(4, 4), dtype=float32)
tf.Tensor(
[[ 0.17399107  0.11753978 -0.12453113  0.12452833]
 [ 0.15034667  0.14374283 -0.13104458  0.12340852]
 [ 0.15133183  0.1557766  -0.12694012  0.13310549]
 [ 0.20472121  0.14842433 -0.09335863  0.14770529]], shape=(4, 4), dtype=float32)
tf.Tensor(
[[-2.0486636]
 [-2.0129147]
 [-2.0172217]
 [-2.0437934]], shape=(4, 1), dtype=float32)


In [9]:
model.epsilon = 0
done = False
episode_reward = 0
start, dest = env.give_start_dest()
ob_user = [start[0], start[1], dest[0], dest[1]]
prev_steps_assist = []
prev_steps_assist = give_prev_steps(prev_steps_assist, steps)
step = 0

while not done and step<max_steps:
    ob_user, prev_steps_assist, reward_user, reward_assist, done = model.step(ob_user, prev_steps_assist)
    episode_reward+=reward_user
    step+=1
    print(ob_user)
    
print(done)

[0.4, 0.8, 0.6, 0.2]
[0.4, 0.7, 0.6, 0.2]
[0.4, 0.6, 0.6, 0.2]
[0.4, 0.5, 0.6, 0.2]
[0.4, 0.4, 0.6, 0.2]
[0.4, 0.3, 0.6, 0.2]
[0.5, 0.3, 0.6, 0.2]
[0.5, 0.2, 0.6, 0.2]
[0.6, 0.2, 0.6, 0.2]
1


In [10]:
reached = 0
for i in range(1000):
    model.epsilon = 0
    done = False
    episode_reward = 0
    start, dest = env.give_start_dest()
    ob_user = [start[0], start[1], dest[0], dest[1]]
    prev_steps_assist = []
    prev_steps_assist = give_prev_steps(prev_steps_assist, steps)
    step = 0

    while not done and step<max_steps:
        ob_user, prev_steps_assist, reward_user, reward_assist, done = model.step(ob_user, prev_steps_assist)
        episode_reward+=reward_user
        step+=1
#         print(ob_user)
        if done:
            reached += 1
#     print(done)

print(reached)

978


In [11]:
ob_user, action_user, reward_user, next_ob_user, ob_assist, action_assist,\
reward_assist, next_ob_assist, done, importance, indices = model.sample_exp()
input_C = np.vstack(tuple([model.env_cell_mapping]*128))
ob_assist

array([[[ 0. ,  1. ,  0. ,  0. ,  1. ,  0.3],
        [ 0. ,  1. ,  0. ,  0. ,  1. ,  0.3],
        [ 0. ,  1. ,  0. ,  0. ,  1. ,  0.3],
        [ 0. ,  1. ,  0. ,  0. ,  1. ,  0.3]],

       [[ 0. ,  1. ,  0. ,  0. ,  0.2,  0.9],
        [ 0. ,  0. ,  1. ,  0. ,  0.3,  0.9],
        [ 0. ,  1. ,  0. ,  0. ,  0.3,  0.8],
        [ 0. ,  0. ,  1. ,  0. ,  0.4,  0.8]],

       [[ 0. ,  1. ,  0. ,  0. ,  1. ,  0.3],
        [ 0. ,  1. ,  0. ,  0. ,  1. ,  0.3],
        [ 0. ,  1. ,  0. ,  0. ,  1. ,  0.3],
        [ 0. ,  1. ,  0. ,  0. ,  1. ,  0.3]],

       ...,

       [[ 0. ,  1. ,  0. ,  0. ,  0.2,  0.8],
        [ 0. ,  1. ,  0. ,  0. ,  0.3,  0.8],
        [ 0. ,  1. ,  0. ,  0. ,  0.4,  0.8],
        [ 0. ,  0. ,  1. ,  0. ,  0.5,  0.8]],

       [[ 0. ,  1. ,  0. ,  0. ,  0.4,  0.6],
        [ 0. ,  0. ,  1. ,  0. ,  0.5,  0.6],
        [ 0. ,  1. ,  0. ,  0. ,  0.5,  0.5],
        [ 0. ,  0. ,  1. ,  0. ,  0.6,  0.5]],

       [[ 0. ,  0. ,  0. ,  0. , -1. , -1. ],
        [ 0

In [12]:
np.argmax(model.assist_model([ob_assist, input_C]), axis = 1) + 1

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)